# 하이브리드 자동 라벨링

이 노트북은 Computer Vision(CV)과 Large Language Model(LLM)을 결합한 하이브리드 자동 라벨링을 수행합니다.

## 개요

하이브리드 라벨링은:
- **CV 기반 분석**: CLIP 모델을 사용한 이미지 유사도 검색
- **LLM 기반 분석**: GPT-4 Vision을 활용한 텍스트 기반 분석
- **결과 융합**: 두 방법의 결과를 가중치 기반으로 결합하여 더 높은 정확도 달성

**원본 코드**: `backend/core/providers/aws/hybrid/aws_hybrid_auto_labeler.py`


In [ ]:
import os
import yaml
from pathlib import Path
from PIL import Image

# 프로젝트 루트
PROJECT_ROOT = Path.cwd()
print(f"프로젝트 루트: {PROJECT_ROOT}")


## 설정 로드


In [ ]:
# 설정 파일 로드
config_path = PROJECT_ROOT / "backend" / "configs" / "default.yaml"

if config_path.exists():
    with open(config_path, 'r', encoding='utf-8') as f:
        config = yaml.safe_load(f)
    print("✅ 설정 파일 로드 완료")
else:
    print("⚠️  설정 파일을 찾을 수 없습니다. 기본 설정을 사용합니다.")
    config = {
        "data": {
            "icons_dir": "data/outputs/aws/icons",
            "taxonomy_csv": "data/aws/taxonomy/aws_resources_models.csv",
            "images_dir": "data/images",
            "output_dir": "data/outputs"
        },
        "cv": {
            "clip_name": "ViT-B-32",
            "clip_pretrained": "laion2b_s34b_b79k"
        },
        "llm": {
            "provider": "openai",
            "api_key": os.getenv("OPENAI_API_KEY"),
            "vision_model": "gpt-4o"
        },
        "hybrid": {
            "cv_weight": 0.6,
            "llm_weight": 0.4,
            "fusion_method": "weighted",
            "iou_threshold": 0.5,
            "confidence_threshold": 0.3
        }
    }


## 하이브리드 오토라벨러 초기화


In [ ]:
import sys
sys.path.insert(0, str(PROJECT_ROOT))

from backend.core.providers.aws.hybrid import AWSHybridAutoLabeler

# API 키 확인
if not config.get("llm", {}).get("api_key") and not os.getenv("OPENAI_API_KEY"):
    print("⚠️  OPENAI_API_KEY 환경변수가 설정되지 않았습니다.")
    print("   export OPENAI_API_KEY='your-api-key'")
else:
    if not config.get("llm", {}).get("api_key"):
        config["llm"]["api_key"] = os.getenv("OPENAI_API_KEY")

# 하이브리드 오토라벨러 생성
try:
    labeler = AWSHybridAutoLabeler(config)
    print("✅ 하이브리드 오토라벨러 초기화 완료")
    print(f"   - CV 가중치: {config['hybrid']['cv_weight']}")
    print(f"   - LLM 가중치: {config['hybrid']['llm_weight']}")
    print(f"   - 융합 방법: {config['hybrid']['fusion_method']}")
except Exception as e:
    print(f"❌ 초기화 실패: {e}")
    print("   필요한 패키지가 설치되어 있는지 확인하세요.")


## 단일 이미지 분석


In [ ]:
# 분석할 이미지 경로
image_path = PROJECT_ROOT / config["data"]["images_dir"]
test_images = list(image_path.glob("*.png")) + list(image_path.glob("*.jpg"))

if test_images:
    test_image = test_images[0]
    print(f"분석할 이미지: {test_image.name}")
    
    # 이미지 분석
    try:
        result = labeler.analyze_image(test_image)
        print(f"\n✅ 분석 완료")
        print(f"   - 감지된 객체 수: {len(result.detections)}")
        print(f"   - 처리 시간: {result.processing_time:.2f}초")
        print(f"   - 분석 방법: {result.method}")
        
        # 상위 5개 결과 출력
        for i, det in enumerate(result.detections[:5], 1):
            print(f"\n   {i}. {det.label}")
            print(f"      - 신뢰도: {det.confidence:.3f}")
            print(f"      - 바운딩 박스: ({det.bbox.x}, {det.bbox.y}, {det.bbox.width}, {det.bbox.height})")
    except Exception as e:
        print(f"❌ 분석 실패: {e}")
else:
    print("⚠️  분석할 이미지를 찾을 수 없습니다.")
    print(f"   이미지 디렉터리: {image_path}")


## 배치 분석


In [ ]:
# 여러 이미지 배치 분석
if test_images and len(test_images) > 1:
    batch_images = test_images[:5]  # 최대 5개만
    print(f"배치 분석: {len(batch_images)}개 이미지")
    
    try:
        batch_result = labeler.analyze_batch(batch_images)
        print(f"\n✅ 배치 분석 완료")
        print(f"   - 총 이미지 수: {len(batch_result.results)}")
        print(f"   - 총 처리 시간: {batch_result.total_time:.2f}초")
        print(f"   - 평균 처리 시간: {batch_result.total_time / len(batch_result.results):.2f}초/이미지")
        
        # 결과 요약
        total_detections = sum(len(r.detections) for r in batch_result.results)
        print(f"   - 총 감지 객체 수: {total_detections}")
    except Exception as e:
        print(f"❌ 배치 분석 실패: {e}")
else:
    print("⚠️  배치 분석할 이미지가 부족합니다.")


## 결과 시각화


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def visualize_detections(image_path: Path, result, max_detections: int = 10):
    """감지 결과 시각화"""
    img = Image.open(image_path)
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    ax.imshow(img)
    ax.axis('off')
    
    # 상위 N개만 표시
    detections = sorted(result.detections, key=lambda x: x.confidence, reverse=True)[:max_detections]
    
    for det in detections:
        # 바운딩 박스 그리기
        rect = patches.Rectangle(
            (det.bbox.x, det.bbox.y),
            det.bbox.width,
            det.bbox.height,
            linewidth=2,
            edgecolor='red',
            facecolor='none'
        )
        ax.add_patch(rect)
        
        # 라벨 표시
        label_text = f"{det.label} ({det.confidence:.2f})"
        ax.text(
            det.bbox.x,
            det.bbox.y - 5,
            label_text,
            bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7),
            fontsize=10,
            color='black'
        )
    
    plt.title(f"하이브리드 분석 결과: {len(detections)}개 감지")
    plt.tight_layout()
    return fig

# 결과 시각화
if test_images:
    try:
        result = labeler.analyze_image(test_images[0])
        fig = visualize_detections(test_images[0], result)
        plt.show()
    except Exception as e:
        print(f"⚠️  시각화 실패: {e}")
